In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.fft import rfft, rfftfreq
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv("final_data.csv.gz")

In [3]:
print(df.columns.to_list())

['timestamp', 'Acceleration x (m/s^2)', 'Acceleration y (m/s^2)', 'Acceleration z (m/s^2)', 'amplitude', 'frequency', 'Gyroscope x (rad/s)', 'Gyroscope y (rad/s)', 'Gyroscope z (rad/s)', 'Illuminance (lx)', 'Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)', 'Latitude (°)', 'Longitude (°)', 'Height (m)', 'Velocity (m/s)', 'Direction (°)', 'Horizontal Accuracy (m)', 'Vertical Accuracy (m)', 'Magnetic field x (µT)', 'Magnetic field y (µT)', 'Magnetic field z (µT)', 'Pressure (hPa)', 'Distance (cm)', 'Common time (s)', 'Activity', 'Mood', 'Arousal ', 'Social engagement ', 'Noise Level', 'Concentration Level']


In [4]:
## creating additional columns summarizing columns 

# computing magnitude (Euclidean norm) of 3-D vector for acc, gyro & magn

sensor_axes = {
    "acc":  ['Acceleration x (m/s^2)',
             'Acceleration y (m/s^2)',
             'Acceleration z (m/s^2)'],
    "gyro": ['Gyroscope x (rad/s)',
             'Gyroscope y (rad/s)',
             'Gyroscope z (rad/s)'],
    "linacc": ['Linear Acceleration x (m/s^2)',
               'Linear Acceleration y (m/s^2)',
               'Linear Acceleration z (m/s^2)'],
    "mag":  ['Magnetic field x (µT)',
             'Magnetic field y (µT)',
             'Magnetic field z (µT)'],
}

for name, axes in sensor_axes.items():
    if all(col in df.columns for col in axes):
        df[f"{name}_mag"] = np.linalg.norm(df[axes].values, axis=1)
    else:
        missing = [c for c in axes if c not in df.columns]
        print(f"⚠️  Skipped {name}_mag — missing columns: {missing}")

print("now have:", [c for c in df.columns if c.endswith('_mag')])

# remove the original acc, linacc & magfield from df? 

now have: ['acc_mag', 'gyro_mag', 'linacc_mag', 'mag_mag']


In [22]:
df.to_csv("feature_engineered_final_data.csv", index=False)

In [23]:
df_sens = pd.read_csv("feature_engineered_final_data.csv")
df_sens["timestamp"] = pd.to_datetime(df_sens["timestamp"])

# 16-row questionnaire table
df_q    = pd.read_csv("feature_engineered_final_data.csv")
timestamps = pd.to_datetime(df_q["timestamp"])
df_q["timestamp"] = pd.to_datetime(df_q["timestamp"])

FS        = 407 # sampling rate (Hz) found during EDA
WIN_SEC   = 30  # look-back window before each EMA
GPS_BAD   = 50  # horiz-accuracy threshold (m)
MAG_SAT   = 300 # saturation μT
N_CLUST   = 3   # GPS k-means cluster count
win = pd.Timedelta(seconds=WIN_SEC)

In [24]:
anchor = pd.to_datetime(df_q["timestamp"]).min()     
sec = pd.to_numeric(df_sens["Common time (s)"], errors="coerce")

sec_offset = sec.iloc[0]       
aligned_sec = sec - sec_offset  

df_sens["timestamp"] = anchor + pd.to_timedelta(aligned_sec, unit="s")


In [26]:
## helper functions 

def haversine(lat1, lon1, lat2, lon2):
    R = 6_371_000  # metres
    lat1, lon1, lat2, lon2 = map(np.radians, (lat1, lon1, lat2, lon2))
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

def dominant_freq(sig, fs):
    yf = np.abs(rfft(sig))
    xf = rfftfreq(len(sig), 1/fs)
    return xf[np.argmax(yf[1:])]     

def band_power(sig, fs, low, high):
    yf = np.abs(rfft(sig))
    xf = rfftfreq(len(sig), 1/fs)
    mask = (xf >= low) & (xf < high)
    return yf[mask].sum()

def orientation_angles(ax, ay, az):
    pitch = np.arctan2(-ax, np.sqrt(ay**2 + az**2))*180/np.pi
    roll  = np.arctan2( ay, az )*180/np.pi
    return pitch, roll

In [27]:
## computation of GPS clusters 

coords = df_sens[["Latitude (°)", "Longitude (°)"]].dropna()
kmeans = KMeans(n_clusters=N_CLUST, random_state=42).fit(coords)
df_sens["loc_cluster"] = kmeans.predict(df_sens[["Latitude (°)", "Longitude (°)"]])

In [28]:
def extract_window_features(w):
    feat = {}
        
    # motion stats
    for col in ["acc_mag", "linacc_mag", "gyro_mag"]:
        feat[f"{col}_mean"]  = w[col].mean()
        feat[f"{col}_std"]   = w[col].std()
        feat[f"{col}_q25"]   = w[col].quantile(.25)
        feat[f"{col}_q75"]   = w[col].quantile(.75)
        feat[f"{col}_cv"]    = w[col].std() / (w[col].mean() + 1e-9)

    # jerks
    jerk = np.abs(np.diff(w["acc_mag"])) * FS
    feat["jerk_mean"] = jerk.mean()

    # frequency fingerprints
    dom = dominant_freq(w["acc_mag"].values, FS)
    feat["acc_dom_freq"] = dom
    feat["acc_band0_3"]  = band_power(w["acc_mag"], FS, 0, 3)
    feat["acc_band3_10"] = band_power(w["acc_mag"], FS, 3, 10)

    # orientation metrics
    pitch, roll = orientation_angles(w["Acceleration x (m/s^2)"],
                                     w["Acceleration y (m/s^2)"],
                                     w["Acceleration z (m/s^2)"])
    flat_ratio = (np.abs(pitch) < 10).mean()
    feat["flat_ratio"] = flat_ratio
    flips = np.sum(np.sign(pitch[:-1]) != np.sign(pitch[1:]))
    feat["orient_flips"] = flips

    # environmental deltas
    feat["light_delta"]     = w["Illuminance (lx)"].iloc[-1] - w["Illuminance (lx)"].iloc[0]
    feat["pressure_delta"]  = w["Pressure (hPa)"].iloc[-1]   - w["Pressure (hPa)"].iloc[0]
    feat["pocket_ratio"]    = (w["Distance (cm)"] == 5).mean()

    # GPS mobility
    lat, lon = w["Latitude (°)"].values, w["Longitude (°)"].values
    dist = haversine(lat[:-1], lon[:-1], lat[1:], lon[1:]).sum()
    feat["dist_m"]       = dist
    feat["stop_ratio"]   = (w["Velocity (m/s)"].fillna(0) < 0.3).mean()
    feat["loc_cluster"]  = w["loc_cluster"].mode().iat[0]

    # quality flags
    feat["gps_bad"]       = (w["Horizontal Accuracy (m)"] > GPS_BAD).any()
    feat["mag_saturation"] = (w["mag_mag"] > MAG_SAT).any()

    return feat

In [29]:
df_q.columns    = df_q.columns.str.strip()      
df_sens.columns = df_sens.columns.str.strip()
print(df_q.columns.tolist())

['timestamp', 'Acceleration x (m/s^2)', 'Acceleration y (m/s^2)', 'Acceleration z (m/s^2)', 'amplitude', 'frequency', 'Gyroscope x (rad/s)', 'Gyroscope y (rad/s)', 'Gyroscope z (rad/s)', 'Illuminance (lx)', 'Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)', 'Latitude (°)', 'Longitude (°)', 'Height (m)', 'Velocity (m/s)', 'Direction (°)', 'Horizontal Accuracy (m)', 'Vertical Accuracy (m)', 'Magnetic field x (µT)', 'Magnetic field y (µT)', 'Magnetic field z (µT)', 'Pressure (hPa)', 'Distance (cm)', 'Common time (s)', 'Activity', 'Mood', 'Arousal', 'Social engagement', 'Noise Level', 'Concentration Level', 'acc_mag', 'gyro_mag', 'linacc_mag', 'mag_mag']


In [30]:
rows = []
win = pd.Timedelta(seconds=WIN_SEC)

for idx, q in df_q.iterrows():
    t_q = q["timestamp"]

    w = df_sens[
        (df_sens["timestamp"] >= t_q - win) &
        (df_sens["timestamp"] <  t_q)
    ]

    # ── Safety check ─────────────────────────────────────────────
    if w.empty:
        print(f"⚠️  window empty for EMA {idx} ({t_q}) — filling NaNs")
        feats = {k: np.nan for k in [
            "acc_mag_mean", "acc_mag_std", "gyro_mag_mean"]}
    else:
        feats = extract_window_features(w)
    # ─────────────────────────────────────────────────────────────

    # H. cross-modal
    feats["noisy_motion"]         = (q["Noise Level"] >= 3) * feats.get("acc_mag_mean", 0)
    feats["social_high_arousal"] = ((q["Social engagement"] >= 2) * q["Arousal"])
    # I. temporal deltas
    if idx > 0:
        for col in ["Mood", "Arousal", "Concentration Level"]:
            feats[f"Δ{col}"] = q[col] - df_q.loc[idx - 1, col]
    else:
        feats.update({"ΔMood": 0, "ΔArousal": 0, "ΔConcentration Level": 0})

    # K. circadian features
    hour = t_q.hour + t_q.minute / 60
    feats["tod_sin"] = np.sin(2 * np.pi * hour / 24)
    feats["tod_cos"] = np.cos(2 * np.pi * hour / 24)

    # raw EMA predictors + target
    feats.update({
        "Mood": q["Mood"],
        "Arousal": q["Arousal"],
        "Social engagement": q["Social engagement"],
        "Noise Level": q["Noise Level"],
        "Concentration Level": q["Concentration Level"],
        "timestamp": t_q,
    })

    rows.append(feats)

feature_df = (
    pd.DataFrame(rows)
      .set_index("timestamp")
      .sort_index()
)

print("Final feature table:", feature_df.shape)
feature_df.head()

⚠️  window empty for EMA 0 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 1 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 2 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 3 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 4 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 5 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 6 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 7 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 8 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 9 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 10 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 11 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 12 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 13 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 14 (1970-01-01 00:00:00) — filling NaNs
⚠️  window empty for EMA 15 (1970-0

/var/folders/lc/9q2gf4dn4snfnltmj87764500000gn/T/ipykernel_70409/1835825859.py:14: RuntimeWarning: Mean of empty slice.
  feat["jerk_mean"] = jerk.mean()
/Users/aylinogras/Desktop/MLQS/ML_for_Sensor_data-1/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: attempt to get argmax of an empty sequence

In [90]:
df.to_csv("feature_engineered_final_data.csv", index=False)

In [91]:
existing = pd.read_csv("feature_engineered_final_data.csv")
existing["timestamp"] = pd.to_datetime(existing["timestamp"])
feature_df_reset = (feature_df.reset_index().rename(columns={"Timestamp": "timestamp"}))
updated = (existing.merge(feature_df_reset,on="timestamp",how="left"))
updated.to_csv("feature_engineered_final_data.csv", index=False)
print("Updated file written:", updated.shape)

Updated file written: (416297, 51)
